In [1]:
import os
import re
print(os.getcwd())
os.chdir('..')

/home/mikolaj/supervision_hack_kiid/notebooks


In [13]:
from lib.pdf_process import split_text_into_sections
import spacy
from pdfminer.high_level import extract_text

file = 'data/filtered/2022-09-21_bps2060.pdf'

nlp = spacy.load("pl_core_news_lg", exclude=["parser"])
nlp.enable_pipe("senter")

cele_i_polityka = split_text_into_sections(extract_text(file), file)[1]
cele_i_polityka = cele_i_polityka[cele_i_polityka.find('\n')+1:]
doc = nlp(cele_i_polityka)

if cele_i_polityka.__contains__("cel"):
    cel = ' '.join([sent for sent in doc.sents if sent.__contains__("cel")])
else:
    cel = list(doc.sents)[0]


print(cel)




Celem  inwestycyjnym  BPS  Emerytura  PPK  2060  jest  wzrost  wartości  aktywów  tego 
Subfunduszu  w  wyniku  wzrostu  wartości  lokat.  Subfundusz  jest  subfunduszem 
zdefiniowanej daty w rozumieniu art. 39 ustawy z 4.10.2018 r. o pracowniczych planach 
kapitałowych.  Zdefiniowaną  datą  dla  Subfunduszu  jest  2060  r.  


In [20]:
nlp =spacy.load("pl_core_news_lg", exclude=["parser"])
nlp.enable_pipe("senter")
def get_cel(row):
    text = row['cele_i_polityka']
    doc = nlp(text)
    for it, cel in enumerate(doc.sents):
        return it

    return doc.sents

def get_ncel(row):
    text = row['cele_i_polityka']
    doc = nlp(text)
    polityka = ' '.join([polityka for i, polityka in enumerate(doc.sents) if i!=0])
    return polityka

In [21]:
get_cel({'dupa':cele_i_polityka})

Celem  inwestycyjnym  BPS  Emerytura  PPK  2060  jest  wzrost  wartości  aktywów  tego 
Subfunduszu  w  wyniku  wzrostu  wartości  lokat.  Subfundusz  jest  subfunduszem 
zdefiniowanej daty w rozumieniu art. 39 ustawy z 4.10.2018 r. o pracowniczych planach 
kapitałowych.  Zdefiniowaną  datą  dla  Subfunduszu  jest  2060  r.  

In [12]:
print(list(doc.sents))

[Celem  inwestycyjnym  BPS  , Emerytura  PPK  2060  jest  wzrost  wartości  aktywów  tego 
Subfunduszu  w  wyniku  wzrostu  wartości  lokat.,  , Subfundusz  jest  subfunduszem 
zdefiniowanej daty w rozumieniu art. 39 ustawy z 4.10.2018 r. o pracowniczych planach 
kapitałowych.  , Zdefiniowaną  datą  dla  Subfunduszu  jest  2060  r.  , Oznacza  to,  że 
Subfundusz  zmienia  swoją  politykę  inwestycyjną  w  czasie,  dążąc  do  zwiększania 
bezpieczeństwa  inwestycyjnego  w  miarę  zbliżania  się  do  zdefiniowanej  daty.,  , Dobór 
lokat  jest  zgodny  z  ustaloną  dla  danego  okresu  alokacją  aktywów  z  uwzględnieniem 
konieczności  dokonywania  stopniowej  zmiany  udziałów  części  udziałowej  i  części 
dłużnej zgodnie z założeniami Subfunduszu., Oznacza to stopniową zmianę polegającą 
na  zmniejszaniu  części  lokat  zainwestowanych  w  instrumenty  charakteryzujące  się 
najwyższym  poziomem  ryzyka  na  rzecz  bardziej  konserwatywnych  lokat  związanych  
z instrumentami o niż

In [82]:
print(cele_i_polityka.replace('  ',' '))

Cele i polityka inwestycyjna 

Celem inwestycyjnym Subfunduszu, jest wzrost wartości Aktywów Subfunduszu w 
wyniku wzrostu wartości lokat. Fundusz nie gwarantuje osiągnięcia celu inwestycyj-
nego Subfunduszu. 

kryteria wskazane w art. 93 i 93a Ustawy o Funduszach Inwestycyjnych. 
Subfundusz może wykorzystywać instrumenty pochodne zarówno w celu ogranicze-
nia ryzyka inwestycyjnego, jak i w celu zapewnienia sprawnego zarządzania portfe-
lem inwestycyjnym. 

Fundusz może inwestować od 0% do 100% Aktywów Subfunduszu w instrumenty 
udziałowe, tj.: akcje oraz instrumenty finansowe o podobnym charakterze takie jak 
w szczególności: prawa poboru, prawa do akcji, warranty subskrypcyjne, kwity 
depozytowe, których bazą są akcje (lub inne prawa majątkowe inkorporujące prawa 
z akcji), obligacje zamienne na akcje, a także inne zbywalne papiery wartościowe 
inkorporujące prawa majątkowe odpowiadające prawom z akcji. 

Subfundusz realizuje nabycia i odkupienia jednostek uczestnictwa w każdym dniu 

In [87]:
from spacy.matcher import Matcher
pattern = [{"LOWER":"jednostek"}, {"LOWER":"uczestnictwa"}, {}, {}, {}]
CZOIJUDICT = {'dzień': 'D', 'tydzień': 'W', 'kwartał': 'Q', 'miesiąc': 'M', 'dziennie':'D', 'codziennie': 'D', 'cotygodniowo': 'W', 'kwartalnie': 'Q', 'miesięcznie': 'M'}
def get_czioju(row):
    cele_i_polityka = row['cele_i_polityka']
    doc = nlp(cele_i_polityka)
    matcher = Matcher(nlp.vocab)
    czioju = []
    matcher.add('test_pat', [pattern])
    matches = matcher(doc)
    for match in matches:
        czioju.append(doc[match[1]:match[2]])
    if len(czioju) == 1:
        return CZOIJUDICT.get(czioju[0], None)
    elif len(czioju) == 0:
        return None
    else:
        return CZOIJUDICT.get(czioju[-1], None)


In [ ]:
pattern = [{"LOWER":"nie"}, {"LEMMA":"wypłaca"}, {"OP":"?"}, {}]

def get_dywidenda(row):
    matcher = Matcher(nlp.vocab)
    cele_i_polityka = row['cele_i_polityka']
    doc = nlp(cele_i_polityka)
    dyws = []
    matcher.add('dywidenda_pattern', pattern)
    matches = matcher(doc)
    for match in matches:
        dyws.append(doc[match[1]:match[2]])
    if len(dyws) > 0:
        return False
    return True

In [89]:
matches


[(18324528750915213551, 164, 169)]

In [90]:
doc[164:169]

jednostek uczestnictwa w każdym dniu

In [43]:
spacy.explain("")

/home/mikolaj/.venv/lib/python3.8/site-packages/spacy/glossary.py:19: UserWarning: [W118] Term 'BEDZIE' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))
